# *How* del framework anidado de Tamara Munzner en Altair

**Profesores:** Hernán Valdivieso y Daniela Flores.

**Ayudantes**: Daniela Concha y Francisca Ibarra.

Instalar librería `itables` (la usaremos en la sección de Interactions)

In [1]:
!pip install itables

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.2 MB/s eta 0:00:00


Importar librerías

In [2]:
import pandas as pd
from vega_datasets import data
import altair as alt

Los datos a utilizar en este notebook corresponden a la información de diferentes autos.

In [3]:
cars = data.cars()
# Actualizar columna Year para solo considerar el año.
cars["Year"] = cars["Year"].apply(lambda x:x.year)

# Eliminar toda fila que le falte algún dato.
cars = cars.dropna().reset_index(drop=True)

cars.head(10)

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970,USA
5,ford galaxie 500,15.0,8,429.0,198.0,4341,10.0,1970,USA
6,chevrolet impala,14.0,8,454.0,220.0,4354,9.0,1970,USA
7,plymouth fury iii,14.0,8,440.0,215.0,4312,8.5,1970,USA
8,pontiac catalina,14.0,8,455.0,225.0,4425,10.0,1970,USA
9,amc ambassador dpl,15.0,8,390.0,190.0,3850,8.5,1970,USA


# Visual encodings

## Arrange

### Express

Codificamos dos variables cuantitativas usando conjuntamente los canales de posición horizontal y vertical.

In [4]:
alt.Chart(cars).mark_point().encode(
    x="Miles_per_Gallon",
    y="Horsepower",
)

alt.Chart(...)

### Separate
Un mapa de calor es un ejemplo de cómo se pueden separar los datos en 2 ejes. Por ejemplo, en la visualización siguiente, separamos los datos en una matriz de dos dimensiones Origen y Año, para que así cada celda represente el promedio de millas por galón de combustible según el origen y el año de la celda en cuestión

In [5]:
alt.Chart(cars).mark_rect().encode(
    y=alt.Y('Origin'),
    x=alt.X('Year:N'),
    color="mean_miles_per_gallon:Q"
).transform_aggregate(
    mean_miles_per_gallon='mean(Miles_per_Gallon)',
    groupby=['Origin', 'Year']
)

alt.Chart(...)

### Order

Podemos ordenar el eje Y de forma alfabética (comportamiento por defecto en Altair)

In [6]:
usa_cars = cars[cars["Origin"] == "USA"]
usa_cars = usa_cars.sort_values("Miles_per_Gallon", ascending=False)
usa_cars = usa_cars.head(10)
alt.Chart(usa_cars).mark_bar().encode(
    y="Name",
    x="Miles_per_Gallon",
)

alt.Chart(...)

Otra opción es ordenar según el valor del eje X (millas por galón de bencina, en nuestro caso)

In [7]:
usa_cars = cars[cars["Origin"] == "USA"]
usa_cars = usa_cars.sort_values("Miles_per_Gallon", ascending=False)
usa_cars = usa_cars.head(10)
alt.Chart(usa_cars).mark_bar().encode(
    y=alt.Y("Name", sort="x"),
    x="Miles_per_Gallon"
)

alt.Chart(...)

# Interactions


## Manipulate

### Change

Una forma de cambio en el tiempo es ordenar las filas según el valor de una columna en particular. La librería `itables` nos permite hacer esto al visualizar un DataFrame de Pandas

In [8]:
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [9]:
cars

Name  Miles_per_Gallon  Cylinders  Displacement  \
0    chevrolet chevelle malibu              18.0          8         307.0   
1            buick skylark 320              15.0          8         350.0   
2           plymouth satellite              18.0          8         318.0   
3                amc rebel sst              16.0          8         304.0   
4                  ford torino              17.0          8         302.0   
..                         ...               ...        ...           ...   
387            ford mustang gl              27.0          4         140.0   
388                  vw pickup              44.0          4          97.0   
389              dodge rampage              32.0          4         135.0   
390                ford ranger              28.0          4         120.0   
391                 chevy s-10              31.0          4         119.0   

     Horsepower  Weight_in_lbs  Acceleration  Year  Origin  
0         130.0           3504          12.0  1970     USA  
1         165.0           3693          11.5  1970     USA  
2         150.0           3436          11.0  1970     USA  
3         150.0           3433          12.0  1970     USA  
4         140.0           3449          10.5  1970     USA  
..          ...            ...           ...   ...     ...  
387        86.0           2790          15.6  1982     USA  
388        52.0           2130          24.6  1982  Europe  
389        84.0           2295          11.6  1982     USA  
390        79.0           2625          18.6  1982     USA  
391        82.0           2720          19.4  1982     USA  

[392 rows x 9 columns]

### Select

#### Select con click

Podemos seleccionar uno de los autos haciendo click en el punto que lo representa.

In [10]:
single = alt.selection_single()

alt.Chart(cars).mark_circle(size=100).encode(
    x="Horsepower",
    y="Miles_per_Gallon",
    color=alt.condition(single, "Origin", alt.value("lightgray"))
).add_selection(
    single
)

alt.Chart(...)

#### Select con hover

Similarmente, podemos seleccionar uno de los autos con hover.

In [ ]:
single = alt.selection_single(on="mouseover", nearest=True)

alt.Chart(cars).mark_circle(size=100).encode(
    x="Horsepower:Q",
    y="Miles_per_Gallon:Q",
    color=alt.condition(single, "Origin", alt.value("lightgray"))
).add_selection(
    single
)

alt.Chart(...)

## Navigate

### Zoom con Altair

Ahora vamos a agregar zoom

In [ ]:
alt.Chart(cars).mark_point().encode(
    x="Miles_per_Gallon",
    y="Horsepower",
    color="Origin",
).interactive() # Una línea hace la magia!!

alt.Chart(...)

## Facet

### Juxtapose: Vistas coordinadas con Altair

Ahora, intentemos coordinar vistas.

In [ ]:
# Estas dos variables nos permiten definir si queremos los labels de los valores de los ejes y sus títulos.
tick_axis = alt.Axis(labels=True, domain=False, ticks=True) # Queremos ver todo
tick_axis_notitle = alt.Axis(labels=True, domain=False, ticks=True, title="") # No queremos el título

# Creamos un gráfico de dispersión donde un eje será el atributo "Miles_per_Gallon" y otro eje será "Horsepower"
# Agregamos un tooltip que mostrará la información del nombre y año del motor.
points = alt.Chart(cars).mark_point().encode(
    x=alt.X("Miles_per_Gallon", axis=tick_axis_notitle),
    y=alt.Y("Horsepower", axis=tick_axis_notitle),
    color="Origin",
    )

# Para el gráfico que va en el eje X no queremos mostrar el título en el eje y.
x_ticks = alt.Chart(cars).mark_tick().encode(
    alt.X("Miles_per_Gallon", axis=tick_axis),  # Queremos que el título del eje X sí aparezca.
    alt.Y("Origin", axis=tick_axis_notitle),  # Ponemos la variable que elimina los títulos.
    color="Origin"
)

# Para el gráfico que va en el eje Y no queremos mostrar el título en el eje X.
y_ticks = alt.Chart(cars).mark_tick().encode(
    alt.X("Origin", axis=tick_axis_notitle),
    alt.Y("Horsepower", axis=tick_axis),
    color="Origin"
)

# Se disponen los gráficos en columnas y filas. Las barras verticales | definen las columnas y los & definen filas.
# Una columna será "y_ticks" y la otra columna tendrá 2 filas;
# La primera fila tendrá "points" y la segunda tendrá "x_ticks".
y_ticks | (points & x_ticks)

alt.HConcatChart(...)

Ahora, qué tal si agregamos un selector de los datos para enfatizarlos cuando se incluyen en la selección

In [ ]:
# Creamos un objeto de selección que permite seleccionar un intervalo de datos
brush = alt.selection(type="interval")   ### NUEVA LINEA

# Estas dos variables nos permiten definir si queremos los labels de los valores de los ejes y sus títulos.
tick_axis = alt.Axis(labels=False, domain=False, ticks=False) # No queremos ver los labels
tick_axis_notitle = alt.Axis(labels=False, domain=False, ticks=False, title="") # No queremos ver los labels ni título

# Creamos un gráfico de dispersión donde un eje será el atributo "Miles_per_Gallon" y otro eje será "Horsepower"
# El color estará condicionado al objeto selección. En caso de seleccionar el dato, usará la columna "Origin" para
# definir su color. En otro caso será gris.
# Agregamos un tooltip que mostrará la información del nombre y año del motor.
# Finalmente agregamos el selector (brush) para que uno pueda seleccionar datos en su gráfico.
points = alt.Chart(cars).mark_point().encode(
    x=alt.X("Miles_per_Gallon", axis=tick_axis_notitle),
    y=alt.Y("Horsepower", axis=tick_axis_notitle),
    color=alt.condition(brush, "Origin", alt.value("grey")),  ### NUEVA LINEA
    tooltip=["Name", "Year"]
    ).add_selection(brush)   ### NUEVA LINEA

# Para el gráfico que va en el eje X no queremos mostrar el título en el eje y. Vamos a hacer que este gráfico no tenga
# el selector de brush. Por lo tanto, no podemos seleccionar datos desde ese gráfico, solo desde el de dispersión.
x_ticks = alt.Chart(cars).mark_tick().encode(
    alt.X("Miles_per_Gallon", axis=tick_axis),  # Queremos que el título del eje X sí aparezca.
    alt.Y("Origin", axis=tick_axis_notitle),  # Ponemos la variable que elimina los títulos.
    color=alt.condition(brush, "Origin", alt.value("lightgrey"))   ### NUEVA LINEA
)

# Para el gráfico que va en el eje Y no queremos mostrar el título en el eje X. Vamos a hacer que este gráfico no tenga
# el selector de brush. Por lo tanto, no podemos seleccionar datos desde ese gráfico, solo desde el de dispersión.
y_ticks = alt.Chart(cars).mark_tick().encode(
    alt.X("Origin", axis=tick_axis_notitle),
    alt.Y("Horsepower", axis=tick_axis),
    color=alt.condition(brush, "Origin", alt.value("lightgrey"))   ### NUEVA LINEA
)

# Se disponen los gráficos en columnas y filas. Las barras verticales | definen las columnas y los & definen filas.
# Una columna será "y_ticks" y la otra columna tendrá 2 filas;
# La primera fila tendrá "points" y la segunda tendrá "x_ticks".
y_ticks | (points & x_ticks)

alt.HConcatChart(...)

Pero el selector no solo puede alterar el color, tambien puede ser una especie de filtro.

In [ ]:
# Creamos un objeto de selección que permite seleccionar un intervalo de datos
brush = alt.selection(type="interval")

points = alt.Chart(cars).mark_point().encode(
    x=alt.X("Miles_per_Gallon"),
    y=alt.Y("Horsepower"),
    color=alt.condition(brush, "Origin", alt.value("grey")),
    tooltip=["Name", "Year"]
    ).add_selection(brush)

# Creamos un segundo gráfico de dispersión donde un eje será el atributo "Acceleration" y otro eje será "Weight_in_lbs"
# El color será el país de origen, pero agrgaremos un filtro que estará condicionado al selector definido antes.
points_2 = alt.Chart(cars).mark_point().encode(
    x=alt.X("Acceleration"),
    y=alt.Y("Weight_in_lbs"),
    color="Origin",
    tooltip=["Name", "Year"]
    ).transform_filter(brush) ### NUEVA LINEA Lo importante es "transform_filter"

# Es una fila con 2 columna.
points | points_2

alt.HConcatChart(...)

Finalmente, podemos exportar la visualización interactiva en un HTML.

In [ ]:
(points | points_2 ).save("chart.html")

In [ ]:
import IPython
IPython.display.HTML(filename="chart.html")

## Reduce

### Filter

#### Filtrar con la leyenda


Altair nos permite agregar interactividad a la leyenda de los gráficos, lo que facilita el filtrado de los datos. A continuación, utilizaremos la leyenda para mostrar solo los datos cuyo origen coincida con el que *clickeemos* en la leyenda.


In [ ]:
selection = alt.selection_multi(fields=["Origin"], bind="legend") # Campo sobre el que aplicaremos el filtro
alt.Chart(cars).mark_point().encode(
    x="Miles_per_Gallon",
    y="Horsepower",
    color="Origin",
    opacity=alt.condition(selection, alt.value(1), alt.value(0)) # Si el dato coincide con el clickeado, la opacidad es 1. En otro caso 0 (así no se muestra el dato)
).add_selection(
    selection
)

alt.Chart(...)

#### Dropdown con Altair

Para seleccionar datos de forma sencilla en nuestras visualizaciones, Altair provee la funcionalidad de dropdown. A continuación, se puede ver un ejemplo aplicado al dataset que estamos trabajando en este notebook. Con el dropdown en cuestión, podemos filtrar los datos según el lugar al que pertenecen.

In [ ]:
input_dropdown = alt.binding_select(options=["Europe","Japan","USA"], name="Origin") # Declaramos el dropdown y sus opciones
selection = alt.selection_single(fields=["Origin"], bind=input_dropdown) # Creamos una selección enlazada a nuestro dropdown

alt.Chart(cars).mark_point().encode(
    x="Horsepower:Q",
    y="Miles_per_Gallon:Q",
    color="Origin:N",
).add_selection(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

Al interactuar con el ejemplo anterior, podemos ver que no es posible volver a mostrar todos los datos una vez que cambiamos el valor seleccionado en el dropdown. Para arreglar esto, agregamos la opción None a los posibles valores del dropdown.

In [ ]:
input_dropdown = alt.binding_select(options=[None, "Europe","Japan","USA"], labels=["All","Europe","Japan","USA"], name="Origin") # Declaramos el dropdown y sus opciones
selection = alt.selection_single(fields=["Origin"], bind=input_dropdown) # Creamos una selección enlazada a nuestro dropdown

alt.Chart(cars).mark_point().encode(
    x="Horsepower:Q",
    y="Miles_per_Gallon:Q",
    color="Origin:N",
    tooltip="Name:N"
).add_selection(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

#### Slider con Altair

Altair también cuenta con un deslizador que permite filtrar valores o cambiar la forma en que se muestra nuestra visualización. En este ejemplo, usaremos un slider para modificar el tamaño de los puntos que representan cada auto.

In [11]:
slider = alt.binding_range(min=0, max=200, step=1, name="size")
selector = alt.selection_single(
    fields=["size"],
    bind=slider,
    init={"size": 1.0}
)

alt.Chart(cars).transform_calculate(
    size=selector.size
).mark_point().encode(
    x="Miles_per_Gallon:Q",
    y="Horsepower:Q",
    size=alt.Size("size:Q", scale=None),
).add_selection(
    selector
)

alt.Chart(...)

### Aggregate


Altair también nos permite realizar transformaciones sobre los datos a graficar, sin tener que realizar estas operaciones en librerías como Pandas.

In [ ]:
alt.Chart(cars).mark_bar().encode(
    y="Year:O",
    x="average(Acceleration):Q"
)

alt.Chart(...)

Podemos probar también otras transformaciones, como contar cuántos autos hay en el dataset para cada origen:

In [ ]:
alt.Chart(cars).mark_bar().encode(
    y="Origin:O",
    x="count(Origin):Q"
)

alt.Chart(...)

También podemos obtener los mínimos y máximos de los valores de aceleración por origen, y compararlos en un gráfico.

In [ ]:
chart_max = alt.Chart(cars).mark_bar().encode(
    y="Origin:O",
    x="max(Acceleration):Q",
)

chart_min = alt.Chart(cars).mark_bar().encode(
    y="Origin:O",
    x="min(Acceleration):Q",
    color=alt.value("blue"),
)

alt.layer(chart_max, chart_min)


alt.LayerChart(...)

## Embed

### Tooltip con Altair

Vamos a agregar _tooltip_ para tener mas info de cada ítem.

In [ ]:
alt.Chart(cars).mark_point().encode(
    x="Miles_per_Gallon",
    y="Horsepower",
    color="Origin",
    tooltip=["Name", "Year", "Miles_per_Gallon"]  # Una línea hace la magia!!
)

alt.Chart(...)

# Material adicional

* https://towardsdatascience.com/how-to-create-interactive-and-elegant-plot-with-altair-8dd87a890f2a
* https://docs.heavy.ai/data-science/interactive-data-exploration-with-altair
* https://medium.com/analytics-vidhya/interactive-data-viz-using-altair-873139771fe2

# Fuentes utilizadas en este notebook

* Documentacion de Altair: https://altair-viz.github.io/gallery/index.html#interactive-charts
* Interacciones con Altair: https://colab.research.google.com/github/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb#scrollTo=NWVWj-hYQqcx